In [2]:
# %% [markdown]
# # 📊 Análisis Exploratorio del Mercado Laboral de Data Science en España
# ## Descubriendo insights y tendencias

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
from collections import Counter
import ast

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# %%
# Cargar datos limpios
df = pd.read_csv("../data/processed/jobs_cleaned.csv")

# Convertir skills de string a lista
df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) and x != '[]' else [])

print(f"📊 Dataset: {len(df)} ofertas × {len(df.columns)} columnas")
df.head()

# %% [markdown]
# ## 1️⃣ Panorama General del Mercado

# %%
# KPIs principales
total_jobs = len(df)
avg_salary = df['salary_avg'].mean()
median_salary = df['salary_avg'].median()
cities = df['city'].nunique()
companies = df['company'].nunique()
ai_jobs = df['is_ai_related'].sum()

# Crear dashboard de KPIs
fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number",
    value = total_jobs,
    title = {"text": "Total Ofertas"},
    domain = {'x': [0, 0.2], 'y': [0.7, 1]}
))

fig.add_trace(go.Indicator(
    mode = "number",
    value = avg_salary,
    number = {'prefix': "€", 'valueformat': ",.0f"},
    title = {"text": "Salario Promedio"},
    domain = {'x': [0.25, 0.45], 'y': [0.7, 1]}
))

fig.add_trace(go.Indicator(
    mode = "number",
    value = cities,
    title = {"text": "Ciudades"},
    domain = {'x': [0.5, 0.7], 'y': [0.7, 1]}
))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = ai_jobs,
    delta = {'reference': total_jobs/2, 'relative': False},
    title = {"text": "Ofertas IA/ML"},
    domain = {'x': [0.75, 0.95], 'y': [0.7, 1]}
))

fig.update_layout(
    title="📈 Mercado Laboral de Data Science en España - KPIs",
    height=300
)

fig.show()

# %% [markdown]
# ## 2️⃣ Distribución de Roles

# %%
# Distribución de categorías de roles
role_counts = df['role_category'].value_counts()

fig = px.bar(
    x=role_counts.values,
    y=role_counts.index,
    orientation='h',
    title="💼 Distribución de Roles de Data Science",
    labels={'x': 'Número de Ofertas', 'y': 'Categoría'},
    color=role_counts.values,
    color_continuous_scale='Blues',
    text=role_counts.values
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.show()

# %%
# Roles por nivel de experiencia
role_seniority = pd.crosstab(df['role_category'], df['seniority'])

fig = px.bar(
    role_seniority,
    title="📊 Distribución de Roles por Nivel de Experiencia",
    labels={'value': 'Número de Ofertas', 'variable': 'Nivel'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_layout(
    xaxis_title="Categoría de Rol",
    yaxis_title="Número de Ofertas",
    legend_title="Nivel de Experiencia",
    height=500
)

fig.show()

# %% [markdown]
# ## 3️⃣ Análisis Geográfico

# %%
# Top 15 ciudades
top_cities = df['city'].value_counts().head(15)

fig = px.bar(
    x=top_cities.index,
    y=top_cities.values,
    title="🏙️ Top 15 Ciudades con Más Ofertas de Data Science",
    labels={'x': 'Ciudad', 'y': 'Número de Ofertas'},
    color=top_cities.values,
    color_continuous_scale='Viridis'
)

fig.update_layout(
    xaxis_tickangle=-45,
    showlegend=False,
    height=500
)

fig.show()

# %%
# Distribución de roles por ciudad (Top 5 ciudades)
top5_cities = df['city'].value_counts().head(5).index
df_top5 = df[df['city'].isin(top5_cities)]

city_role_dist = pd.crosstab(df_top5['city'], df_top5['role_category'])

fig = px.bar(
    city_role_dist,
    title="💼 Distribución de Roles en las Principales Ciudades",
    labels={'value': 'Número de Ofertas', 'variable': 'Tipo de Rol'},
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(height=500, xaxis_title="Ciudad", yaxis_title="Ofertas")
fig.show()

# %% [markdown]
# ## 4️⃣ Análisis de Skills

# %%
# Top 20 skills más demandadas
all_skills = [skill for skills_list in df['skills'] if skills_list for skill in skills_list]
skill_counts = Counter(all_skills)
top_skills = dict(skill_counts.most_common(20))

fig = px.bar(
    x=list(top_skills.values()),
    y=list(top_skills.keys()),
    orientation='h',
    title="🔥 Top 20 Skills Más Demandadas",
    labels={'x': 'Número de Ofertas', 'y': 'Skill'},
    color=list(top_skills.values()),
    color_continuous_scale='Reds',
    text=list(top_skills.values())
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(showlegend=False, height=600)
fig.show()

# %%
# Skills por categoría de rol (Top 5 roles)
top_roles = df['role_category'].value_counts().head(5).index

skills_by_role = {}
for role in top_roles:
    role_skills = [skill for skills_list in df[df['role_category']==role]['skills'] 
                   if skills_list for skill in skills_list]
    skills_by_role[role] = Counter(role_skills).most_common(10)

# Crear subplots
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=[f"{role}" for role in top_roles],
    specs=[[{"type": "bar"}, {"type": "bar"}, {"type": "bar"}],
           [{"type": "bar"}, {"type": "bar"}, {"type": "xy"}]]
)

positions = [(1,1), (1,2), (1,3), (2,1), (2,2)]

for idx, role in enumerate(top_roles):
    skills = [s[0] for s in skills_by_role[role]]
    counts = [s[1] for s in skills_by_role[role]]
    
    row, col = positions[idx]
    fig.add_trace(
        go.Bar(y=skills, x=counts, orientation='h', name=role, showlegend=False),
        row=row, col=col
    )

fig.update_layout(
    title_text="🎯 Top 10 Skills por Categoría de Rol",
    height=800,
    showlegend=False
)

fig.show()

# %% [markdown]
# ## 5️⃣ Análisis Salarial

# %%
# Filtrar solo ofertas con salario
df_salary = df[df['salary_avg'].notna()].copy()

print(f"💰 Análisis basado en {len(df_salary)} ofertas con información salarial")

# %%
# Distribución de salarios
fig = px.histogram(
    df_salary,
    x='salary_avg',
    nbins=30,
    title="💶 Distribución de Salarios en Data Science",
    labels={'salary_avg': 'Salario Anual (€)', 'count': 'Número de Ofertas'},
    color_discrete_sequence=['#3498db']
)

fig.add_vline(x=df_salary['salary_avg'].mean(), line_dash="dash", 
              line_color="red", annotation_text=f"Media: {df_salary['salary_avg'].mean():,.0f}€")
fig.add_vline(x=df_salary['salary_avg'].median(), line_dash="dash", 
              line_color="green", annotation_text=f"Mediana: {df_salary['salary_avg'].median():,.0f}€")

fig.update_layout(height=500)
fig.show()

# %%
# Salarios por categoría de rol
salary_by_role = df_salary.groupby('role_category')['salary_avg'].agg(['mean', 'median', 'count'])
salary_by_role = salary_by_role[salary_by_role['count'] >= 10].sort_values('mean', ascending=False)

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Salario Promedio',
    x=salary_by_role.index,
    y=salary_by_role['mean'],
    text=salary_by_role['mean'].apply(lambda x: f"{x:,.0f}€"),
    textposition='outside'
))

fig.add_trace(go.Bar(
    name='Salario Mediano',
    x=salary_by_role.index,
    y=salary_by_role['median'],
    text=salary_by_role['median'].apply(lambda x: f"{x:,.0f}€"),
    textposition='outside'
))

fig.update_layout(
    title="💼 Salarios por Tipo de Rol",
    xaxis_title="Categoría",
    yaxis_title="Salario Anual (€)",
    barmode='group',
    height=500
)

fig.show()

# %%
# Salarios por nivel de experiencia
salary_by_seniority = df_salary.groupby('seniority')['salary_avg'].agg(['mean', 'median', 'count'])
salary_by_seniority = salary_by_seniority[salary_by_seniority['count'] >= 10].sort_values('mean', ascending=False)

fig = px.bar(
    x=salary_by_seniority.index,
    y=salary_by_seniority['mean'],
    title="📊 Salarios por Nivel de Experiencia",
    labels={'x': 'Nivel', 'y': 'Salario Promedio (€)'},
    color=salary_by_seniority['mean'],
    color_continuous_scale='Greens',
    text=salary_by_seniority['mean'].apply(lambda x: f"{x:,.0f}€")
)

fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.show()

# %%
# Salarios por ciudad (Top 10 con suficientes datos)
salary_by_city = df_salary.groupby('city')['salary_avg'].agg(['mean', 'count'])
salary_by_city = salary_by_city[salary_by_city['count'] >= 10].sort_values('mean', ascending=False).head(10)

fig = px.bar(
    x=salary_by_city.index,
    y=salary_by_city['mean'],
    title="🏙️ Salarios por Ciudad (Top 10)",
    labels={'x': 'Ciudad', 'y': 'Salario Promedio (€)'},
    color=salary_by_city['mean'],
    color_continuous_scale='Blues',
    text=salary_by_city['mean'].apply(lambda x: f"{x:,.0f}€")
)

fig.update_traces(textposition='outside')
fig.update_layout(showlegend=False, xaxis_tickangle=-45, height=500)
fig.show()

# %%
# Boxplot de salarios por nivel
fig = px.box(
    df_salary,
    x='seniority',
    y='salary_avg',
    title="📦 Distribución Salarial por Nivel de Experiencia",
    labels={'seniority': 'Nivel', 'salary_avg': 'Salario (€)'},
    color='seniority',
    points='outliers'
)

fig.update_layout(showlegend=False, height=500)
fig.show()

# %% [markdown]
# ## 6️⃣ Análisis de IA/ML

# %%
# Comparación IA vs No-IA
ai_comparison = df.groupby('is_ai_related').agg({
    'id': 'count',
    'salary_avg': 'mean'
}).rename(columns={'id': 'count'})

ai_comparison.index = ['No-IA', 'IA/ML']

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Número de Ofertas", "Salario Promedio"),
    specs=[[{"type": "pie"}, {"type": "bar"}]]
)

# Pie chart de distribución
fig.add_trace(
    go.Pie(labels=ai_comparison.index, values=ai_comparison['count'], 
           marker_colors=['#95a5a6', '#e74c3c']),
    row=1, col=1
)

# Bar chart de salarios
fig.add_trace(
    go.Bar(x=ai_comparison.index, y=ai_comparison['salary_avg'],
           marker_color=['#95a5a6', '#e74c3c'],
           text=ai_comparison['salary_avg'].apply(lambda x: f"{x:,.0f}€" if pd.notna(x) else "N/A"),
           textposition='outside'),
    row=1, col=2
)

fig.update_layout(
    title_text="🤖 Comparación: Ofertas de IA/ML vs Resto",
    height=400,
    showlegend=False
)

fig.show()

# %%
# Tendencia temporal de ofertas IA/ML
df['created'] = pd.to_datetime(df['created'])
df_temporal = df.set_index('created').resample('M').agg({
    'is_ai_related': ['sum', 'count']
})

df_temporal.columns = ['AI_Jobs', 'Total_Jobs']
df_temporal['AI_Percentage'] = (df_temporal['AI_Jobs'] / df_temporal['Total_Jobs'] * 100)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=df_temporal.index, y=df_temporal['AI_Jobs'], 
               name="Ofertas IA/ML", line=dict(color='red', width=3)),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_temporal.index, y=df_temporal['Total_Jobs'], 
               name="Total Ofertas", line=dict(color='blue', width=2, dash='dash')),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_temporal.index, y=df_temporal['AI_Percentage'], 
               name="% IA/ML", line=dict(color='green', width=2)),
    secondary_y=True,
)

fig.update_layout(
    title_text="📈 Evolución Temporal de Ofertas IA/ML",
    height=500
)

fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Número de Ofertas", secondary_y=False)
fig.update_yaxes(title_text="Porcentaje IA/ML (%)", secondary_y=True)

fig.show()

# %% [markdown]
# ## 7️⃣ Top Empresas que Contratan

# %%
# Top 20 empresas
top_companies = df['company'].value_counts().head(20)

fig = px.bar(
    x=top_companies.values,
    y=top_companies.index,
    orientation='h',
    title="🏢 Top 20 Empresas que Más Contratan en Data Science",
    labels={'x': 'Número de Ofertas', 'y': 'Empresa'},
    color=top_companies.values,
    color_continuous_scale='Teal',
    text=top_companies.values
)

fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(showlegend=False, height=700)
fig.show()

# %% [markdown]
# ## 8️⃣ Resumen de Insights Clave

# %%
print("=" * 70)
print("🎯 INSIGHTS CLAVE DEL ANÁLISIS".center(70))
print("=" * 70)

print("\n1️⃣ PANORAMA GENERAL:")
print(f"   • {total_jobs} ofertas activas de Data Science en España")
print(f"   • Salario promedio: {avg_salary:,.0f}€/año")
print(f"   • {ai_jobs} ({ai_jobs/total_jobs*100:.1f}%) son roles de IA/ML")

print("\n2️⃣ ROLES MÁS DEMANDADOS:")
for i, (role, count) in enumerate(df['role_category'].value_counts().head(3).items(), 1):
    print(f"   {i}. {role}: {count} ofertas ({count/total_jobs*100:.1f}%)")

print("\n3️⃣ CIUDADES TOP:")
for i, (city, count) in enumerate(df['city'].value_counts().head(5).items(), 1):
    print(f"   {i}. {city}: {count} ofertas")

print("\n4️⃣ SKILLS MÁS VALORADAS:")
for i, (skill, count) in enumerate(skill_counts.most_common(5), 1):
    print(f"   {i}. {skill}: {count} menciones ({count/len(df)*100:.1f}%)")

if len(df_salary) > 0:
    print("\n5️⃣ SALARIOS:")
    print(f"   • Rango típico: {df_salary['salary_avg'].quantile(0.25):,.0f}€ - {df_salary['salary_avg'].quantile(0.75):,.0f}€")
    print(f"   • Nivel Senior: {df_salary[df_salary['seniority']=='Senior']['salary_avg'].mean():,.0f}€")
    print(f"   • Nivel Junior: {df_salary[df_salary['seniority']=='Junior']['salary_avg'].mean():,.0f}€")

print("\n" + "=" * 70)

📊 Dataset: 2056 ofertas × 17 columnas


💰 Análisis basado en 781 ofertas con información salarial


                    🎯 INSIGHTS CLAVE DEL ANÁLISIS                     

1️⃣ PANORAMA GENERAL:
   • 2056 ofertas activas de Data Science en España
   • Salario promedio: 84,577€/año
   • 1256 (61.1%) son roles de IA/ML

2️⃣ ROLES MÁS DEMANDADOS:
   1. Data/ML Engineer: 544 ofertas (26.5%)
   2. Data Scientist: 430 ofertas (20.9%)
   3. Data Analyst: 348 ofertas (16.9%)

3️⃣ CIUDADES TOP:
   1. España: 703 ofertas
   2. Madrid: 598 ofertas
   3. Barcelona: 511 ofertas
   4. Málaga: 59 ofertas
   5. Valencia: 18 ofertas

4️⃣ SKILLS MÁS VALORADAS:
   1. Python: 72 menciones (3.5%)
   2. Azure: 69 menciones (3.4%)
   3. SQL: 54 menciones (2.6%)
   4. AWS: 38 menciones (1.8%)
   5. Spark: 30 menciones (1.5%)

5️⃣ SALARIOS:
   • Rango típico: 80,000€ - 80,000€
   • Nivel Senior: 85,468€
   • Nivel Junior: 86,667€

